### FEA GC Registry Demo Notebook

This notebook walks the user through some common operations that may be performed against the registry, and will evolve as more features are added.

Preliminary steps:
* Follow the first few steps of the `README` to setup the poetry environment and make sure that the kernel of this notebook us using the venv created there 
* Run `make db.seed` in the terminal first to populate the database with GC bundles and example entities
* Once you've built the registry images, run `docker compose up` in a terminal window. 

If at any point you want to restart your progress, you can run `make db.reset` to clear the database and then proceed from the steps above.

In [1]:
import pandas as pd
import datetime
import httpx
import logging.config

from gc_registry.account.schemas import AccountBase, AccountWhitelist
from gc_registry.certificate.schemas import GranularCertificateQuery, GranularCertificateTransfer, GranularCertificateCancel
from gc_registry.device.models import DeviceBase
from gc_registry.core.models.base import DeviceTechnologyType, EnergySourceType

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

There are quite a few middleware handlers in the FastAPI and HTTPX setup, so to declutter the notebook we disable most of them here so that only the relevant loggers are active. You can view to the HTTP logs in the terminal that the docker cluster is running in.

In [ ]:
client = httpx.Client(base_url="http://localhost:8000")

LOGGING_CONFIG = {
    "version": 1,
    "handlers": {
        "default": {
            "class": "logging.StreamHandler",
            "formatter": "http",
            "stream": "ext://sys.stderr"
        }
    },
    "formatters": {
        "http": {
            "format": "%(levelname)s [%(asctime)s] %(name)s - %(message)s",
            "datefmt": "%Y-%m-%d %H:%M:%S",
        }
    },
    'loggers': {}
}

logging.config.dictConfig(LOGGING_CONFIG)

### Login for Access Token

User access management is handled through a basic OAuth2 setup, with the seeding process creating an Admin user with the default name `Admin` and password `admin`.

In [ ]:
token = client.post("/auth/login", data={"username": "a_user@usermail.com", "password": "admin"}).json()

headers = {"Authorization": f"Bearer {token["access_token"]}"}

token

### Creating a Account, and Device

The seeding process will have created some user, account, and device entities automatically to allocate the GC bundles to; these can be viewed with get commands to the integer ID of those entities.

In [ ]:
user_from_db = client.get("user/1", headers=headers)

user_from_db.json()

In [ ]:
account_from_db = client.get("account/1", headers=headers)

account_from_db.json()

Entities can be created through the appropriate data model, ensuring that field validation is performed ahead of submission to the API. We're going to create another account that also belongs to the same user to demonstrate transfering GC bundles between accounts. 

Admin, Production, and Trading User roles can create accounts, but only Admin and Production Users can create devices.

In [ ]:
account = AccountBase(account_name="Target Account", user_ids=[1])

result = client.post("account/create", data=account.model_dump_json(), headers=headers)
result.json()

In [ ]:
# This account whitelists the existing source account so that it in can receive GC bundles from it

whitelist = AccountWhitelist(add_to_whitelist=[1])

whitelist_result = client.post(f"account/update_whitelist/{account.id}", data=whitelist.model_dump_json(), headers=headers)
whitelist_result.json()

In [ ]:
device_from_db = client.get("device/1", headers=headers)

device_from_db.json()

In [ ]:
device_dict = {
    "device_name": "fake_solar_device",
    "grid": "fake_grid",
    "energy_source": EnergySourceType.solar_pv,
    "technology_type": DeviceTechnologyType.solar_pv,
    "meter_data_id": "BMU-ABC",
    "power_mw": 1000,
    "account_id": 1,
    "fuel_source": "solar",
    "location": "USA",
    "operational_date": pd.to_datetime("2020-01-01"),
    "peak_demand": 100,
    "is_storage": False,
    "is_deleted": False,
}

device = DeviceBase(**device_dict)

result = client.post("device/create", data=device.model_dump_json(), headers=headers)
result.json()

### Querying Existing GC Bundles

In [ ]:
# For display purposes, limit the columns to those we're interested in for now
relevant_columns = ["id", "issuance_id", "account_id", "device_id", "production_starting_interval", "certificate_bundle_id_range_start", "certificate_bundle_id_range_end", "bundle_quantity", "certificate_bundle_status", "is_deleted"]

Querying is handled through the `GranularCertificateQuery` object that contains fields against which GC bundles in a single account can be filtered. The workflow for subsequent actions requires that the bundle IDs are passed from a query into the action endpoint, to prevent the user performing actions blindly on certificates returned from a query whose results they can only see once the action has been performed.

In [ ]:
# The user ID must be included to make sure that the user has access to the target accounts. In practice, the user would be automatically filled
# in on the backend by using the authenticated identity of the party making the request, rather than providing the ID as a parameter. 
gc_bundle_query = GranularCertificateQuery(
    user_id=user_from_db.json()["id"],
    source_id=1,
    device_id=1,
)

gc_bundles_from_query = client.post("certificate/query", content=gc_bundle_query.model_dump_json(), headers=headers)

pd.DataFrame(gc_bundles_from_query.json()["granular_certificate_bundles"])[relevant_columns].sort_values("production_starting_interval")

In [ ]:
# Let's narrow down a subset of certificates using a time range
gc_bundle_query = GranularCertificateQuery(
    user_id=user_from_db.json()["id"],
    source_id=1,
    device_id=1,
    certificate_period_start=datetime.datetime(2024, 1, 1, 5, 0, 0),
    certificate_period_end=datetime.datetime(2024, 1, 1, 8, 0, 0)
)

gc_bundles_from_query = client.post("certificate/query", data=gc_bundle_query.model_dump_json(), headers=headers)

pd.DataFrame(gc_bundles_from_query.json()["granular_certificate_bundles"])[relevant_columns].sort_values("production_starting_interval")

### Transfering Bundles

In this example, we're going to take the IDs of the bundles returned from the query above, and transfer half of each of these bundles to the target account we created earlier.

In [ ]:
transfer_action = GranularCertificateTransfer(
    user_id=user_from_db.json()["id"],
    source_id=1,
    target_id=2,
    granular_certificate_bundle_ids = [6, 7, 8],
    certificate_bundle_percentage=0.5
)

response = client.post("certificate/transfer", data=transfer_action.model_dump_json(), headers=headers)

response.json()

We can then view those transferred GC bundles in the target account. Note that the IDs have changed because these are new bundles that have been created by splitting the original target bundles.

In [ ]:
gc_bundle_query = GranularCertificateQuery(
    user_id=user_from_db.json()["id"],
    source_id=2,
)

gc_bundles_from_query = client.post("certificate/query", data=gc_bundle_query.model_dump_json(), headers=headers)

pd.DataFrame(gc_bundles_from_query.json()["granular_certificate_bundles"])[relevant_columns].sort_values("production_starting_interval")

### Cancelling Bundles

The target account can then select those bundles for cancellation using the same workflow as for transfer.

In [ ]:
gc_bundle_ids_to_cancel = pd.DataFrame(gc_bundles_from_query.json()["granular_certificate_bundles"])["id"].tolist()

gc_cancel_action = GranularCertificateCancel(
    user_id=user_from_db.json()["id"],
    source_id=2,
    beneficiary="Future Energy Associates",
    granular_certificate_bundle_ids=gc_bundle_ids_to_cancel
)

response = client.post("certificate/cancel", data=gc_cancel_action.model_dump_json(), headers=headers)

response.json()

We can then view that these GC bundles have been cancelled by looking at their `certificate_bundle_status` fields.

In [ ]:
gc_bundle_query = GranularCertificateQuery(
    user_id=user_from_db.json()["id"],
    source_id=2,
)

gc_bundles_from_query = client.post("certificate/query", data=gc_bundle_query.model_dump_json(), headers=headers)

pd.DataFrame(gc_bundles_from_query.json()["granular_certificate_bundles"])[relevant_columns].sort_values("production_starting_interval")